In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 14:00:13.363027


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

Sure, let's define the parameters, decision variables, objective function, and constraints for this problem:

Parameters:
- \( A \) is the maximum capacity of the mixture tank, which is 80 liters.
- \( B \) is the upper limit for the squared difference between the liters used of each component and the mean liters used, which is 10.

Decision Variables:
- \( x_1 \) is the liters of the first liquid used.
- \( x_2 \) is the liters of the second liquid used.

Objective Function:
The objective is to maximize the quality of the product, which is measured by the difference between the Napierian logarithm of the product of liters used of each component and the cube root of the summation of the first component liters as well as double the second component liters. Therefore, the objective function can be written as:
\[ max \quad f(x_1, x_2) = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2} \]

Constraints:
1. The squared difference between the liters employed of each component and the mean liters utilized of both components should be below ten. This can be written as:
\[ \left(x_1 - \frac{x_1 + x_2}{2}\right)^2 + \left(x_2 - \frac{x_1 + x_2}{2}\right)^2 \leq B \]

2. The mixture tank can only fit 80 l. This can be written as:
\[ x_1 + x_2 \leq A \]

3. Non-negativity constraint, as the liters used cannot be negative:
\[ x_1 \geq 0, \quad x_2 \geq 0 \]

So, the complete mathematical optimization model is:
\[ max \quad f(x_1, x_2) = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2} \]
subject to:
\[ \left(x_1 - \frac{x_1 + x_2}{2}\right)^2 + \left(x_2 - \frac{x_1 + x_2}{2}\right)^2 \leq B \]
\[ x_1 + x_2 \leq A \]
\[ x_1 \geq 0, \quad x_2 \geq 0 \]

In [7]:
print(response_1_text)

Sure, let's define the parameters, decision variables, objective function, and constraints for this problem:

Parameters:
- \( A \) is the maximum capacity of the mixture tank, which is 80 liters.
- \( B \) is the upper limit for the squared difference between the liters used of each component and the mean liters used, which is 10.

Decision Variables:
- \( x_1 \) is the liters of the first liquid used.
- \( x_2 \) is the liters of the second liquid used.

Objective Function:
The objective is to maximize the quality of the product, which is measured by the difference between the Napierian logarithm of the product of liters used of each component and the cube root of the summation of the first component liters as well as double the second component liters. Therefore, the objective function can be written as:
\[ max \quad f(x_1, x_2) = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2} \]

Constraints:
1. The squared difference between the liters employed of each component and the mean liters uti

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

To solve the optimization problem using Pyomo, we will first install the necessary packages. If you haven't installed Pyomo and a suitable solver (like GLPK), you can do so by running:

```sh
pip install pyomo
pip install glpk
```

Now, let's write the Pyomo code for this optimization problem:

```python
import pyomo.environ as pyo
import math

# Define the parameters
A = 80  # Maximum capacity of the mixture tank
B = 10  # Upper limit for the squared difference

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return pyo.log(model.x1 * model.x2) - pow((model.x1 + 2 * model.x2), 1/3)

model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return pyo.square(model.x1 - 0.5 * (model.x1 + model.x2)) + pyo.square(model.x2 - 0.5 * (model.x1 + model.x2)) <= B

model.constraint1 = pyo.Constraint(rule=constraint1)

def constraint2(model):
    return model.x1 + model.x2 <= A

model.constraint2 = pyo.Constraint(rule=constraint2)

# Solve the optimization problem using GLPK solver
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Print the results (decision variables and objective function value)
print(f"x1: {pyo.value(model.x1)}, x2: {pyo.value(model.x2)}, Objective Function Value: {pyo.value(model.obj)}")
```

This code defines the optimization problem using Pyomo, including the parameters, decision variables, objective function, and constraints. It then uses the GLPK solver to solve the problem and prints the optimal solution. Replace `A` and `B` with your own values as needed.

In [11]:
print(response_2_text)

To solve the optimization problem using Pyomo, we will first install the necessary packages. If you haven't installed Pyomo and a suitable solver (like GLPK), you can do so by running:

```sh
pip install pyomo
pip install glpk
```

Now, let's write the Pyomo code for this optimization problem:

```python
import pyomo.environ as pyo
import math

# Define the parameters
A = 80  # Maximum capacity of the mixture tank
B = 10  # Upper limit for the squared difference

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return pyo.log(model.x1 * model.x2) - pow((model.x1 + 2 * model.x2), 1/3)

model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return pyo.square(model.x1 - 0.5 * (model.x1 + model.x2)) + pyo.square(model.x2 - 0.5 * (model.x1 + mo

### Code Executability

In [12]:
import pyomo.environ as pyo
import math

# Define the parameters
A = 80  # Maximum capacity of the mixture tank
B = 10  # Upper limit for the squared difference

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return pyo.log(model.x1 * model.x2) - pow((model.x1 + 2 * model.x2), 1/3)

model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return pyo.square(model.x1 - 0.5 * (model.x1 + model.x2)) + pyo.square(model.x2 - 0.5 * (model.x1 + model.x2)) <= B

model.constraint1 = pyo.Constraint(rule=constraint1)

def constraint2(model):
    return model.x1 + model.x2 <= A

model.constraint2 = pyo.Constraint(rule=constraint2)

# Solve the optimization problem using GLPK solver
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Print the results (decision variables and objective function value)
print(f"x1: {pyo.value(model.x1)}, x2: {pyo.value(model.x2)}, Objective Function Value: {pyo.value(model.obj)}")

ERROR:pyomo.core:Rule failed when generating expression for Constraint constraint1 with index None:
AttributeError: module 'pyomo.environ' has no attribute 'square'
ERROR:pyomo.core:Constructing component 'constraint1' from data=None failed:
    AttributeError: module 'pyomo.environ' has no attribute 'square'


AttributeError: module 'pyomo.environ' has no attribute 'square'

### Solution Correctness

In [15]:
import pyomo.environ as pyo
import math

# Define the parameters
A = 80  # Maximum capacity of the mixture tank
B = 10  # Upper limit for the squared difference

# Create the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def obj_func(model):
    return pyo.log(model.x1 * model.x2) - pow((model.x1 + 2 * model.x2), 1/3)

model.obj = pyo.Objective(rule=obj_func, sense=pyo.maximize)

# Define the constraints
def constraint1(model):
    return (model.x1 - 0.5 * (model.x1 + model.x2))**2 + (model.x2 - 0.5 * (model.x1 + model.x2))**2 <= B

model.constraint1 = pyo.Constraint(rule=constraint1)

def constraint2(model):
    return model.x1 + model.x2 <= A

model.constraint2 = pyo.Constraint(rule=constraint2)

# Solve the optimization problem using GLPK solver
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model)

# Print the results (decision variables and objective function value)
print(f"x1: {pyo.value(model.x1)}, x2: {pyo.value(model.x2)}, Objective Function Value: {pyo.value(model.obj)}")

x1: 42.236067853543105, x2: 37.76393234768411, Objective Function Value: 2.473033919646447
